In [1]:
# Phoebe Waters DVAMI19h

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# pip install pyqt5
from sklearn.preprocessing import KBinsDiscretizer

In [2]:
#fixing dataset
df = pd.read_csv("spambase.csv")

df = df.drop_duplicates()
df = df.reset_index()

In [ ]:
#high percentage = 1
#meadium = 0
#low = -1
# here I made my own quantile function, however there are functions available in dataprocessing libraries.

def descretinization_3(column_list):
    mean = column_list.mean()
    std = column_list.std()

    for index,value in column_list.items():
        if value > mean + std:
            column_list[index] = 1
        elif value < mean - std:
            column_list[index] = -1
        else:
            column_list[index] = 0

In [6]:
# another way to descretecize, binning
est = KBinsDiscretizer(n_bins= 5, encode='ordinal', strategy='uniform')
est.fit(df)
df_matrix = est.transform(df)

In [5]:
print(df)

[[0. 0. 0. ... 0. 0. 4.]
 [0. 0. 0. ... 0. 0. 4.]
 [0. 0. 0. ... 0. 0. 4.]
 ...
 [4. 0. 0. ... 0. 0. 0.]
 [4. 1. 0. ... 0. 0. 0.]
 [4. 0. 0. ... 0. 0. 0.]]


In [ ]:
#split data into training and testing

In [ ]:
# algorithm 4.2 LGG conj


def LGG_conj(instance, main_list):
    list_false_index = []
    list_values = []
    for i in range(instance):
        if instance[i] == main_list[i]:
            list_values.append(main_list[i])
        else:
            list_false_index.append(i)

    return list_false_index,list_values

In [ ]:
#algorithm 4.1 with 4......
main_list = df_matrix[0]
for instance in df_matrix[1:]:
    #compare first and second instance 
    false_index_list, main_list = LGG_conj(instance, main_list)
    for i in false_index_list:
        np.delete(df_matrix,i,1)